<a href="https://colab.research.google.com/github/girinath18/AI_Headshot_RG/blob/headshots/Nokia2API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install pyngrok

In [16]:
from pyngrok import ngrok

In [17]:
ngrok.set_auth_token("2ga6KvgSqbJ8a8UcyMG6EC2AeTJ_3tcyVNAHQsQXkzpxUgu9p")
public_url = ngrok.connect(5000).public_url

In [ ]:
!git clone https://github.com/girinath18/roop.git

fatal: destination path 'roop' already exists and is not an empty directory.


In [ ]:
!pip install flask_ngrok
!pip install onnxruntime
!pip install customtkinter
!pip install torch torchvision
!pip install flask
!pip install pyngrok --upgrade
!pip install --upgrade pyngrok
!pip install -r /content/roop/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring tkinterdnd2-universal: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring onnxruntime: markers 'python_version != "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-coreml: markers 'python_version == "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment


In [ ]:
%cd roop

/content/roop


In [18]:
from flask import Flask, request, jsonify
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

# Default templates folder path
TEMPLATES_FOLDER_PATH = "/content/template/Kings"

def face_swap(source_image_folder, output_base_folder):
    try:
        # Create output folder if it doesn't exist
        output_folder = os.path.join(output_base_folder, "AR(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        # Get list of source image filenames
        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        # Get list of template image filenames
        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")

        # Iterate over source images
        for source_image_name in source_images:
            source_image_path = os.path.join(source_image_folder, source_image_name)

            # Check if the item in the folder is a file
            if os.path.isfile(source_image_path):
                logging.info(f"Processing source image: {source_image_name}")

                processed_image_paths = set()

                for target_image_name in template_images:
                    target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                    # Check if the item in the folder is a file
                    if os.path.isfile(target_image_path):
                        logging.info(f"Processing target image: {target_image_name}")

                        output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                        if output_image_path in processed_image_paths:
                            logging.info(f"Skipping already processed image: {output_image_path}")
                            continue

                        # Run the command for each target image
                        command = (
                            f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                            f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                            f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                            f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                        )
                        logging.info(f"Running command: {command}")

                        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                        stdout, stderr = process.communicate()
                        if process.returncode != 0:
                            logging.error(f"Failed to process target image: {target_image_name}")
                            logging.error(f"Command output: {stderr.decode('utf-8')}")
                        else:
                            logging.info(f"Successfully processed target image: {target_image_name}")
                            logging.info(f"Command output: {stdout.decode('utf-8')}")
                            processed_image_paths.add(output_image_path)

                        # Stop processing if the processed image count equals the template image count
                        if len(processed_image_paths) >= len(template_images):
                            break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        # Create temporary source image folder
        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)

        # Save source images to temporary folder
        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        output_base_folder = "/content"

        face_swap(source_image_folder, output_base_folder)

        # Delete temporary source image folder
        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    # Start ngrok tunnel
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


 * Ngrok tunnel URL: NgrokTunnel: "https://7ef2-34-16-146-201.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Failed to process target image: KG (11).jpg
ERROR:root:Command output: /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(
Segmentation fault (core dumped)

ERROR:root:Failed to process target image: KG (3).jpg
ERROR:root:Command output: /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(
Segmentation fault (core dumped)

ERROR:root:Failed to process target image

In [ ]:
from flask import Flask, request, jsonify
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

# Default templates folder path
TEMPLATES_FOLDER_PATH = "/content/template/doctor"

def face_swap(source_image_folder, output_base_folder):
    try:
        # Create output folder if it doesn't exist
        output_folder = os.path.join(output_base_folder, "(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        # Get list of source image filenames
        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        # Get list of template image filenames
        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")

        # Iterate over source images
        for source_image_name in source_images:
            source_image_path = os.path.join(source_image_folder, source_image_name)

            # Check if the item in the folder is a file
            if os.path.isfile(source_image_path):
                logging.info(f"Processing source image: {source_image_name}")

                # Keep a counter to ensure only one output per template image
                processed_images_count = 0

                for target_image_name in template_images:
                    target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                    # Check if the item in the folder is a file
                    if os.path.isfile(target_image_path):
                        logging.info(f"Processing target image: {target_image_name}")

                        # Run the command for each target image
                        command = (
                            f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                            f'-o "{output_folder}" --keep-fps --execution-threads 14 --many-faces '
                            f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                            f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                        )
                        logging.info(f"Running command: {command}")

                        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                        stdout, stderr = process.communicate()
                        if process.returncode != 0:
                            logging.error(f"Failed to process target image: {target_image_name}")
                            logging.error(f"Command output: {stderr.decode('utf-8')}")
                        else:
                            logging.info(f"Successfully processed target image: {target_image_name}")
                            logging.info(f"Command output: {stdout.decode('utf-8')}")

                        processed_images_count += 1

                        # Stop processing if the processed image count equals the template image count
                        if processed_images_count >= len(template_images):
                            break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        # Create temporary source image folder
        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)

        # Save source images to temporary folder
        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        output_base_folder = "/content"

        face_swap(source_image_folder, output_base_folder)

        # Delete temporary source image folder
        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    # Start ngrok tunnel
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)
